## Detailed Pipeline Execution


### Prior steps: running a search

This notebook can be used to execute, parse and visualize results obtained when using the ReScore pipeline proposed in [_Accurate peptide fragmentation predictions allow data driven approaches to replace and improve upon proteomics search engine scoring functions_](https://www.biorxiv.org/content/10.1101/428805v2).

The first step neede to execute the pipeline is a regular target-decoy search. We start by searching the _Pyrococcus furiosus_ spectra from PRIDE project *PXD001077* against all _Pyrococcus furiosus_ sequences. The `.mgf` and `.fasta` files can be downloaded from [our](http://genesis.ugent.be/uvpublicdata/silvia/PXD001077/Velos005137.mgf) [servers](http://genesis.ugent.be/uvpublicdata/silvia/PXD001077/pyroUniprotAll.fasta) for reproducing these results. The search is done using [MS-GF+](https://github.com/MSGFPlus/msgfplus) as follows:

`java -Xmx28000M -jar MSGFPlus.jar -mod modifications.txr -s Velos005137.mgf -d pyroUniprotAll.fasta -o Velos005137.mzid -t10ppm -tda 1 -m 3 -inst 1 -minLength 8 -minCharge 2 -maxCharge 4 -n 1 -e 1 -addFeatures 1 -protocol 0 -thread 23`

The resulting `.mzid` file can be found [in our servers as well](http://genesis.ugent.be/uvpublicdata/silvia/PXD001077/Velos005137.mzid).

### Prior steps: creating PEPREC file

Normally, to use Percolator, one would execute the following command to obtain a `.pin` file with the default Percolator feature set:

`msgf2pin -P XXX Velos005137.mzid > Velos005137.pin`

This file needs some processing in order for it to be easily parsed, namely:
- when one peptide maps to more than one protein, the protein identifiers are separated by `\t`, which is a problem using most `.tsv`-parsing schemes
- Percolator's `PSMId` does not map directly to the `.mgf` file's `TITLE` field

To tackle both of these issues, we wrote a small script that:
- separates multiple protein mappings with a `|`
- adds a `TITLE` column to the `.pin` file that directly corresponds to the `.mgf`'s `TITLE` field.

This can be found in [mapper](https://github.com/anasilviacs/mapper). The `.pin` file we obtain after executing this script is [also available for download](http://genesis.ugent.be/uvpublicdata/silvia/PXD001077/Velos005137.perc_pin).

The resulting `.pin` file can be used to generate a `PEPREC` file by using the following function:

In [15]:
import json
import pandas as pd
import re

def make_pepfile(path_to_pin, options):
    """
    Read a pin file and create the corresponding pepfile dataframe, which will
    be saved as a MS2PIP PEPREC file.
    
    :param path_to_pin: path to pin file
    :param options: path to config.json
    
    Returns
    :pd.DataFrame pepfile, columns ['TITLE', 'Peptide', 'Charge', 'Label',
        'modifications', 'Proteins']
    """
    
    # parse config.json
    with open(options) as f:
        config = json.load(f)
    
    # read pin file
    pin = pd.read_csv(path_to_pin, sep='\t', header=0)
    
    # parse charge states from the pin file's "ChargeX" columns
    charge_states = []
    for a in pin.columns:
        if a.startswith('Charge'): charge_states.append(a)
        else: continue

    pin.loc[:, 'Charge'] = [None] * len(pin)

    for ch in charge_states:
        value = int(ch.lstrip('Charge'))
        pin.loc[pin[ch]==1, 'Charge'] = value
    
    # initialize the pepfile with the relevant columns from the pin file
    pepfile = pin.loc[:, ['TITLE', 'Peptide', 'Charge', 'Label', 'Proteins']]

    # add modifications column to pepfile; they are parsed from config.json
    # the keys correspond to the UNIMOD keys for each modification
    modifications = {}
    mods = config["ms2pip"]["modifications"]
    for mod in mods:
        modifications[str(mod["unimod_accession"])] = mod["name"]

    # read peptide sequences and write "peptide" and "modifications" columns according to MS2PIP's conventions
    modlist = []
    for _, row in pepfile.iterrows():
        if 'UNIMOD' in row['Peptide']:
            pep = row['Peptide'].split('.')[1]
            mods = re.findall(r'\[([^]]*)\]', pep)
            modstring = ''
            for mod in mods:
                mod = '[' + mod + ']'
                key = mod.split(':')[1].rstrip(']')
                try:
                    # special case for phosphorilations
                    if key == '21':
                        phospholoc = pep[pep.find(mod)-1]
                        modstring += str(pep.find(mod)) + '|' + modifications[key] + phospholoc + '|'
                        pep = pep.replace(mod, '', 1)
                    else:
                        modstring += str(pep.find(mod)) + '|' + modifications[key] + '|'
                        pep = pep.replace(mod, '', 1)
                except:
                    print('Modification not expected: {}'.format(mod))
            modlist.append(modstring.rstrip('|'))
        else:
            modlist.append('')
    pepfile.loc[:, 'modifications'] = modlist
    
    
    # rewrite peptide sequences without the UNIMOD modification ids
    pep_list = []
    for _, row in pepfile.iterrows():
        pep = row['Peptide']
        pep = pep.split('.')[1]
        if 'UNIMOD' in pep:
            mods = re.findall(r'\[([^]]*)\]', pep)
            for mod in mods:
                pep = pep.replace('[' + mod + ']', '', 1)
        pep_list.append(pep)
    pepfile.loc[:, 'peptide'] = pep_list
    
    # write pepfile to a .PEPREC file
    pepfile = pepfile.loc[:, ['TITLE', 'modifications', 'peptide', 'Charge', 'Label', 'Proteins']]
    pepfile.columns = ['spec_id', 'modifications', 'peptide', 'charge', 'Label', 'Proteins']
    pepfile.to_csv(path_to_pin.replace('.perc_pin', '.PEPREC'), sep=' ', index=False)

In [16]:
make_pepfile('steps/Velos005137.perc_pin', '../config.json')

Resulting `.PEPREC` file for reference [here](http://genesis.ugent.be/uvpublicdata/silvia/PXD001077/pyroUniprotAll.PEPREC). You may notice the `.pin` file's extension was changed to  `.perc_pin`, to avoid it being confused with the file that will be generated in the following steps.

## Executing the pipeline

With the `.PEPREC` and `.mgf` files, the ReScore pipeline can be executed:

`python driver.py steps/Velos005137.mgf steps/Velos005137.PEPREC ../config.json`

This should result in a few files being created, namely:
- `Velos005137_HCD_correlations.csv` - correlations between predicted and empirical spectra -- note that these will be low, due to the presence of decoy and false positive PSMs in the data;
- `Velos005137_HCD_pred_and_emp.csv` - predicted and empirical spectrum for each PSM
- `Velos005137_all_features.csv` - all of the spectrum comparison metrics calculated 
- `Velos005137.pin` - pin file with the selected MS2PIP features

The new `.pin` file can now be used with Percolator:

`percolator Velos005137.pin -m Velos005137.pout -M Velos005137.pout_decoys -w Velos005137.weights -U`